In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("asdasdasasdas/garbage-classification")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/garbage-classification


In [2]:
# === Imports & HYPERPARAMS ===
import os, random, zipfile
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import torchvision.utils as vutils

# Hyperparameters
EPOCHS = 800
BATCH_SIZE = 64
IMGSZ = 64
LR = 2e-4
BETA1 = 0.5
Z_DIM = 100
N_GEN_PER_CLASS = 50
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_WORKERS = 2
SEED = 42

In [3]:
# Paths
INPUT_ROOT = Path("/kaggle/input/garbage-classification/Garbage classification")
OUT_ROOT = Path("/content/gan_out")
CHECKPOINT_DIR = OUT_ROOT / "checkpoints"
SAMPLES_DIR = OUT_ROOT / "samples"
AUG_DIR = OUT_ROOT / "gan_augmented"
for d in (OUT_ROOT, CHECKPOINT_DIR, SAMPLES_DIR, AUG_DIR):
    d.mkdir(parents=True, exist_ok=True)

# Reproducibility
random.seed(SEED)
torch.manual_seed(SEED)

In [4]:
# Try to detect the dataset root
img_root = None
if INPUT_ROOT.exists():
    candidate = INPUT_ROOT / "Garbage classification"
    if candidate.exists():
        img_root = candidate
    else:
        subdirs = [p for p in INPUT_ROOT.iterdir() if p.is_dir()]
        found = False
        for s in subdirs:
            if any(c.is_dir() for c in s.iterdir()):
                img_root = s
                found = True
                break
        if not found:
            img_root = INPUT_ROOT
else:
    raise FileNotFoundError(f"Dataset path not found: {INPUT_ROOT}")

print("Using image root:", img_root)

# Transforms: normalize to [-1,1] for tanh output
transform = transforms.Compose([
    transforms.Resize(IMGSZ),
    transforms.CenterCrop(IMGSZ),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

dataset = datasets.ImageFolder(root=str(img_root), transform=transform)
class_names = dataset.classes
num_classes = len(class_names)
print("Classes:", class_names, "num:", num_classes)

dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True,
                        num_workers=NUM_WORKERS, pin_memory=True)

Using image root: /kaggle/input/garbage-classification/Garbage classification/Garbage classification
Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash'] num: 6


In [5]:
# === Model utilities ===
import torch.nn.functional as F

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1 or classname.find('Linear') != -1:
        try:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        except Exception:
            pass
    if hasattr(m, 'bias') and m.bias is not None:
        try:
            nn.init.constant_(m.bias.data, 0)
        except Exception:
            pass

# Generator: 4x4 -> 8 -> 16 -> 32 -> 64 (exact)
class Generator(nn.Module):
    def __init__(self, z_dim, num_classes, ngf=64, nc=3):
        super().__init__()
        self.z_dim = z_dim
        self.num_classes = num_classes
        self.ngf = ngf

        self.project = nn.Linear(z_dim, ngf*8*4*4)  # -> (B, ngf*8,4,4)

        self.label_map_conv = nn.Sequential(
            nn.ConvTranspose2d(num_classes, ngf*8, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(True)
        )

        self.main = nn.Sequential(
            nn.ConvTranspose2d(ngf*16, ngf*8, 4, 2, 1, bias=False),  # 4 -> 8
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),   # 8 -> 16
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),   # 16 -> 32
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*2, nc, 4, 2, 1, bias=False),      # 32 -> 64
            nn.Tanh()
        )

    def forward(self, z, labels):
        b = z.size(0)
        zproj = self.project(z).view(b, self.ngf*8, 4, 4)
        onehot = torch.zeros(b, self.num_classes, device=z.device)
        onehot[torch.arange(b), labels] = 1.0
        onehot = onehot.view(b, self.num_classes, 1, 1)
        labmap = self.label_map_conv(onehot)  # (B, ngf*8, 4,4)
        inp = torch.cat([zproj, labmap], dim=1)  # (B, ngf*16,4,4)
        out = self.main(inp)  # (B,3,64,64)
        return out

# Discriminator: conv stack that reduces 64 -> 1 via final conv
class Discriminator(nn.Module):
    def __init__(self, num_classes, ndf=64, nc=3):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc + num_classes, ndf, 4, 2, 1, bias=False),   # 64 -> 32
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False),              # 32 -> 16
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False),           # 16 -> 8
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*4, ndf*8, 4, 2, 1, bias=False),           # 8 -> 4
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*8, 1, 4, 1, 0, bias=False)                 # 4 -> 1 (B,1,1,1)
        )

    def forward(self, img, labels):
        b = img.size(0)
        onehot = torch.zeros(b, num_classes, device=img.device)
        onehot[torch.arange(b), labels] = 1.0
        onehot = onehot.unsqueeze(-1).unsqueeze(-1)
        labmap = onehot.expand(-1, -1, img.shape[2], img.shape[3])
        inp = torch.cat([img, labmap], dim=1)
        out = self.main(inp)
        out = out.view(out.size(0))  # (B,)
        return out  # logits for BCEWithLogitsLoss

# Instantiate
netG = Generator(Z_DIM, num_classes).to(DEVICE)
netD = Discriminator(num_classes).to(DEVICE)
netG.apply(weights_init)
netD.apply(weights_init)

# Loss & optimizers
criterion = nn.BCEWithLogitsLoss()
optimizerD = optim.Adam(netD.parameters(), lr=LR, betas=(BETA1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=LR, betas=(BETA1, 0.999))

# Resume from latest checkpoint if present
start_epoch = 1
ckpts = sorted(list(CHECKPOINT_DIR.glob("dcgan_epoch*.pth")), key=lambda p: p.stat().st_mtime, reverse=True)
if ckpts:
    latest = ckpts[0]
    try:
        ck = torch.load(latest, map_location=DEVICE)
        if "netG" in ck:
            netG.load_state_dict(ck["netG"])
        if "netD" in ck:
            netD.load_state_dict(ck["netD"])
        start_epoch = int(ck.get("epoch", 0)) + 1
        print("Resumed from", latest, "starting at epoch", start_epoch)
    except Exception as e:
        print("Could not load checkpoint:", e)
else:
    print("No checkpoint found. Starting at epoch 1.")

No checkpoint found. Starting at epoch 1.


In [6]:
# === Training loop ===
from tqdm import tqdm
fixed_noise = torch.randn(num_classes * 8, Z_DIM, device=DEVICE)
fixed_labels = torch.arange(num_classes, device=DEVICE).repeat_interleave(8)

real_label_val = 0.9  # label smoothing
fake_label_val = 0.0

print("Starting training on", DEVICE)
for epoch in range(start_epoch, EPOCHS + 1):
    loop = tqdm(dataloader, desc=f"Epoch {epoch}/{EPOCHS}")
    for i, (real_images, labels) in enumerate(loop):
        netD.zero_grad()
        real_images = real_images.to(DEVICE)
        labels = labels.to(DEVICE)
        b_size = real_images.size(0)

        # Train D on real
        label_real = torch.full((b_size,), real_label_val, device=DEVICE)
        outD_real = netD(real_images, labels)   # (B,)
        errD_real = criterion(outD_real, label_real)
        errD_real.backward()

        # Train D on fake
        noise = torch.randn(b_size, Z_DIM, device=DEVICE)
        fake_labels = torch.randint(0, num_classes, (b_size,), device=DEVICE)
        fake_images = netG(noise, fake_labels)
        label_fake = torch.full((b_size,), fake_label_val, device=DEVICE)
        outD_fake = netD(fake_images.detach(), fake_labels)
        errD_fake = criterion(outD_fake, label_fake)
        errD_fake.backward()
        optimizerD.step()

        # Train G
        netG.zero_grad()
        outD_fake2 = netD(fake_images, fake_labels)   # (B,)
        label_for_gen = torch.full((b_size,), real_label_val, device=DEVICE)
        errG = criterion(outD_fake2, label_for_gen)
        errG.backward()
        optimizerG.step()

        if (i % 100) == 0:
            loop.set_postfix(D=(errD_real + errD_fake).item(), G=errG.item())

    # Save checkpoint
    torch.save({"netG": netG.state_dict(), "netD": netD.state_dict(), "epoch": epoch},
               CHECKPOINT_DIR / f"dcgan_epoch{epoch}.pth")

    # Save sample grid
    with torch.no_grad():
        netG.eval()
        fake_samples = netG(fixed_noise, fixed_labels).cpu()
        netG.train()
    grid = vutils.make_grid((fake_samples + 1.0) / 2.0, nrow=8, padding=2, normalize=False)
    vutils.save_image(grid, SAMPLES_DIR / f"epoch_{epoch}_samples.png")
    print(f"Saved sample grid for epoch {epoch} -> {SAMPLES_DIR / f'epoch_{epoch}_samples.png'}")

print("Training finished. Checkpoints in", CHECKPOINT_DIR, "samples in", SAMPLES_DIR)

Starting training on cuda


Epoch 1/800: 100%|██████████| 40/40 [00:11<00:00,  3.51it/s, D=1.49, G=4.63]


Saved sample grid for epoch 1 -> /content/gan_out/samples/epoch_1_samples.png


Epoch 2/800: 100%|██████████| 40/40 [00:07<00:00,  5.58it/s, D=1.84, G=4.01]


Saved sample grid for epoch 2 -> /content/gan_out/samples/epoch_2_samples.png


...

Saved sample grid for epoch 798 -> /content/gan_out/samples/epoch_798_samples.png


Epoch 799/800: 100%|██████████| 40/40 [00:06<00:00,  5.84it/s, D=1.98, G=3.02]


Saved sample grid for epoch 799 -> /content/gan_out/samples/epoch_799_samples.png


Epoch 800/800: 100%|██████████| 40/40 [00:07<00:00,  5.15it/s, D=0.335, G=4.51]


Saved sample grid for epoch 800 -> /content/gan_out/samples/epoch_800_samples.png
Training finished. Checkpoints in /content/gan_out/checkpoints samples in /content/gan_out/samples


In [7]:
#generate N images per class
from PIL import Image
netG.eval()
batch_g = 64
for cls_idx, cls_name in enumerate(class_names):
    target_dir = AUG_DIR / cls_name
    target_dir.mkdir(parents=True, exist_ok=True)
    generated = 0
    while generated < N_GEN_PER_CLASS:
        bs = min(batch_g, N_GEN_PER_CLASS - generated)
        noise = torch.randn(bs, Z_DIM, device=DEVICE)
        labels = torch.full((bs,), cls_idx, dtype=torch.long, device=DEVICE)
        with torch.no_grad():
            fake = netG(noise, labels).cpu()  # [-1,1]
        fake = (fake + 1.0) / 2.0  # to [0,1]
        for i in range(bs):
            img = transforms.ToPILImage()(fake[i])
            img_path = target_dir / f"gen_{generated + i + 1:05d}.png"
            img.save(img_path)
        generated += bs
    print(f"Saved {N_GEN_PER_CLASS} images to {target_dir}")

Saved 50 images to /content/gan_out/gan_augmented/cardboard
Saved 50 images to /content/gan_out/gan_augmented/glass
Saved 50 images to /content/gan_out/gan_augmented/metal
Saved 50 images to /content/gan_out/gan_augmented/paper
Saved 50 images to /content/gan_out/gan_augmented/plastic
Saved 50 images to /content/gan_out/gan_augmented/trash
